## Dogs vs. Cats
In this competition, you'll write an algorithm to classify whether images contain either a dog or a cat.  This is easy for humans, dogs, and cats. Your computer will find it a bit more difficult.


![](https://storage.googleapis.com/kaggle-competitions/kaggle/3362/media/woof_meow.jpg)

Deep Blue beat Kasparov at chess in 1997.  
Watson beat the brightest trivia minds at Jeopardy in 2011.  
Can you tell Fido from Mittens in 2013?  


## Your task:

1. Train your algorithm on these files and predict the labels (1 = dog, 0 = cat).
2. Deploy your model in herroku ! The user must have the possibility to upload a photo to test the model.

##  Dataset 
````
./dataset/
----> training_set/
--------> dog/
               image1
               image2
               .
               .
--------> cat/
               image1
               image2
               .
               .
----> test_set/
--------> dog/
               image1
               image2
               .
               .
--------> cat/
               image1
               image2
               .
           .
````

To load the dataset uses ImageDataGenerator : 
https://keras.io/preprocessing/image/

In [8]:
# import tensorflow as tf 
# tf.test.gpu_device_name() 


'/device:GPU:0'

In [70]:
# example of loading the cifar10 dataset
from matplotlib import pyplot as plt
from tensorflow.keras.datasets import cifar10
from tensorflow.python.keras import utils
from tensorflow.keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.models import Sequential

from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.preprocessing.image import image
from tensorflow.keras import regularizers
from tensorflow.keras import optimizers 
import tensorflow as tf
import numpy as np

In [10]:
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
# from google.colab import drive

# drive.mount('/content/gdrive')
# root_path = 'gdrive/My Drive/Colab Notebooks/02-Project/Cat_or_dog'  #change dir to your project folder

Mounted at /content/gdrive


In [12]:
# print(root_path)

gdrive/My Drive/Colab Notebooks/02-Project/Cat_or_dog


In [26]:
# create a data generator
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

In [39]:
# load and iterate training dataset
train_it = train_datagen.flow_from_directory('./dataset/training_set/', class_mode='binary',target_size=(64, 64),color_mode="rgb",batch_size=32,shuffle=True, seed=42)

# load and iterate validation dataset
val_it = val_datagen.flow_from_directory('./dataset/single_prediction/', class_mode='binary', batch_size=32,target_size=(64, 64))
# load and iterate test dataset
test_it = test_datagen.flow_from_directory('./dataset/test_set/', class_mode='binary',target_size=(64, 64),color_mode="rgb",
                                       batch_size=32,shuffle=True, seed=42)


Found 8000 images belonging to 2 classes.
Found 4 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [5]:
print(type(train_it))

<class 'keras_preprocessing.image.directory_iterator.DirectoryIterator'>


In [6]:
# confirm the iterator works
batchX, batchy = train_it.next()
print('Batch shape=%s, min=%.3f, max=%.3f' % (batchX.shape, batchX.min(), batchX.max()))

Batch shape=(32, 64, 64, 3), min=0.000, max=1.000


In [7]:
print(type(batchX))

<class 'numpy.ndarray'>


In [21]:
def model_4_layers():
    num_classes =2
 
    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding='same', input_shape=(3,64,64), activation='relu')) # Layer 1
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Conv2D(32,(3, 3), activation='relu' )) # Layer 2
    model.add(Dropout(0.25)) #reducing overfitting and improving the generalization of deep neural networks.
 
    model.add(Conv2D(64, (3, 3), padding='same', activation='relu')) # Layer 3
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(64, (3, 3), padding='same', activation='relu')) # Layer 4
    model.add(Dropout(0.25))
 
    model.add(Flatten())
    model.add(Dense(512,activation='relu' ))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    
    # Compile model 
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

cnn_4 = model_4_layers()

In [22]:
cnn_4.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 64, 64, 32)        896       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 30, 30, 32)        9248      
_________________________________________________________________
dropout_3 (Dropout)          (None, 30, 30, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 30, 30, 64)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 15, 15, 64)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 15, 15, 64)       

In [23]:
# Declare variables
batch_size = 32 # 32 examples in a mini-batch, smaller batch size means more updates in one epoch
num_classes = 1 # number of outputs possible
epochs =  50 # repeat 

In [24]:
STEP_SIZE_TRAIN=train_it.n//train_it.batch_size
STEP_SIZE_VALID=test_it.n//test_it.batch_size
cnn_4.fit_generator(generator=train_it,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=test_it,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=10
)

Epoch 1/10
250/250 [==============================] - 91s 366ms/step - loss: 7.6666 - accuracy: 0.5000 - val_loss: 7.6898 - val_accuracy: 0.4985
Epoch 2/10
250/250 [==============================] - 86s 345ms/step - loss: 7.6666 - accuracy: 0.5000 - val_loss: 7.6898 - val_accuracy: 0.4985
Epoch 3/10
250/250 [==============================] - 93s 372ms/step - loss: 7.6666 - accuracy: 0.5000 - val_loss: 7.6898 - val_accuracy: 0.4985
Epoch 4/10
250/250 [==============================] - 94s 374ms/step - loss: 7.6666 - accuracy: 0.5000 - val_loss: 7.6898 - val_accuracy: 0.4985
Epoch 5/10
250/250 [==============================] - 91s 363ms/step - loss: 7.6666 - accuracy: 0.5000 - val_loss: 7.6898 - val_accuracy: 0.4985
Epoch 6/10
250/250 [==============================] - 92s 369ms/step - loss: 7.6666 - accuracy: 0.5000 - val_loss: 7.6898 - val_accuracy: 0.4985
Epoch 7/10
250/250 [==============================] - 87s 349ms/step - loss: 7.6666 - accuracy: 0.5000 - val_loss: 7.6898 - val_ac

In [64]:
scores4l = cnn_4.evaluate(test_it, verbose=0)
print("Accuracy: %.2f%%" % (scores4l[1]*100))

Accuracy: 50.00%


**Evaluate the model**

In [65]:
cnn_4.evaluate_generator(generator=test_it,
steps=STEP_SIZE_VALID)

[7.689804792404175, 0.4984879]

**Save your model**

In [66]:
cnn_4.save("cnn4.h5")

In [67]:
STEP_SIZE_val=val_it.n//val_it.batch_size
val_it.reset()
pred=cnn_4.predict_generator(val_it,
steps=STEP_SIZE_val)

In [68]:
print(pred)

[]


In [69]:
train_it.class_indices
if pred == 1:
    prediction = 'dog'
else:
    prediction = 'cat'
print(prediction)

cat


In [63]:
test_image = image.load_img('./dataset/single_prediction/cats/cat_or_dog_2.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn_4.predict(test_image)
train_it.class_indices
if result[0][0] == 1:
    prediction = 'dog'
else:
    prediction = 'cat'
print(prediction)

dog
